## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


In [1]:
#Libraries
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageChops
from shapely import geometry
%matplotlib qt 


In [2]:
# Computing the camera calibration matrix and distortion coefficients given a set of chessboard images.
def compute_calibration():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    nx = 9 # the number of inside corners in x
    ny = 6 # the number of inside corners in y
    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('./camera_cal/*.jpg')
    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)
        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
            # cv2.imshow('img',img)
            # cv2.waitKey(100)
    cv2.destroyAllWindows()
    return objpoints, imgpoints


# Applying distortion correction to raw images
def distortion_correction(img, objpoints, imgpoints):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1],None, None)
    undist_img = cv2.undistort(img,  mtx, dist, None, mtx)    
    return undist_img


# Perspective transform
def perspective_transform(img, src_vertices, dst_vertices):
    transform_mat = cv2.getPerspectiveTransform(src_vertices, dst_vertices)
    # Minv = cv2.getPerspectiveTransform(src_vertices, dst_vertices)
    warped = cv2.warpPerspective(img, transform_mat, img_size, flags=cv2.INTER_LINEAR)
    return warped


# x and y sobel Gradients
def mag_gradient(img,sobel_kernel,mag_thresh=(0,255)):
    #convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Sobel x gradiants
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    # Sobel y gradiants
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        #Calculate the gradient magnitude
    # grad_mag = np.sqrt(sobel_x**2 + sobel_y**2)
    grad_mag = np.sqrt(sobel_x**2)
    # Rescale to 8 bit
    scale_factor = np.max(grad_mag)/255 
    grad_mag = (grad_mag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(grad_mag)
    binary_output[(grad_mag >= mag_thresh[0]) & (grad_mag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# S-channel thresholding
def hls_threshold(img, thresh):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_s_output = np.zeros_like(s_channel)
    binary_s_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1

    return binary_s_output

# Histogram
def image_histogram(img):
    bottom_half = img[img.shape[0]//2:,:]
    histogram = np.sum(bottom_half, axis=0)

    return histogram


In [37]:
# compute calibration data.
obj_points, img_points = compute_calibration()

images = glob.glob('./test_images/*.jpg')

thresh=(90, 255)

f, ((ax1, ax2, ax6),(ax3, ax4, ax5)) = plt.subplots(2, 3, figsize=(24, 9))

src_points = [[260,655],[560,470],[730,470],[1040,655]]
src_vertices = np.float32([src_points])
poly_src = geometry.Polygon(src_points)
src_x,src_y = poly_src.exterior.xy

dst_points = [[280,655],[280,0],[1020,0],[1020,655]]
dst_vertices = np.float32([dst_points])
poly_dst = geometry.Polygon(dst_points)
dst_x,dst_y = poly_dst.exterior.xy


for fname in images:
    img = mpimg.imread(fname) #RGB image
    img_size = (img.shape[1], img.shape[0])

    # apply the calibration data to test images
    distortion_correction(img, obj_points, img_points)

    # sobel_x gradient
    mag_binary = mag_gradient(img, 3, mag_thresh = (20,100))
    # HLS color space, S-channel thresholding
    binary_s_output= hls_threshold(img, (170,255))

    # stacked images for visualization
    color_binary = np.dstack(( np.zeros_like(mag_binary), mag_binary, binary_s_output)) * 255

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(mag_binary)
    combined_binary[(binary_s_output == 1) | (mag_binary == 1)] = 255

    # Applying Perspective transform
    warped = perspective_transform(combined_binary, src_vertices, dst_vertices)

    histogram = image_histogram(warped)
    # Plot
    ax1.imshow(img)
    ax1.plot(src_x,src_y, 'r')
    ax1.set_title('Original Image' + fname, fontsize=50)
    ax2.imshow(combined_binary, cmap='gray')
    ax2.set_title('Combined Binary', fontsize=50)
    ax3.imshow(mag_binary, cmap='gray')
    ax3.set_title('Gradient', fontsize=50)
    ax4.plot(dst_x,dst_y, 'b')
    # ax4.plot(histogram)
    ax4.imshow(warped, cmap='gray')
    ax4.set_title('lanes', fontsize=50)
    ax5.plot(histogram)

    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

    # plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.pause(2)
    plt.cla()
    # plt.waitforbuttonpress()


    # Saving the images
    name = "./output_images" + fname.replace('./test_images', '')
    cv2.imwrite( name, warped )

plt.close('all')


